# Olist Data Analyst Challenge

### Background:

###### 
This challenge ensure's the following:
- A working knowledge of data analysis tools and methodologies
- Experience analyzing large data sets
- Preparation and presentation of data

### Prompt:
###### 
We are using 2 data sets (linked below) from Brazillian retailer Olist.
Here are some facts:
 - It’s currently September 1st 2018 (Ignoring September 2018 and after)
 - The company’s inception was January 1st 2017 (Ignore all data before January 2017)
 - Company is US-based, but launched in Brazil (which is why some information is in Portuguese, inlcuding currency)
 - Summarizes the seller funnel from marketing sign up to launching products on the platform
 - Please provide Customer LTV [Helpful Site with Examples of Equations (geckoboard.com)](https://www.geckoboard.com/best-practice/kpi-examples/)
 - Summarizes the current state of the business
 - What has performance been monthly
 - What are the best selling categories
 - Predicts future revenue for the next 12 months for order volume and revenue

### Data Sources:
###### 
- [Marketing Funnel Data Set](https://www.kaggle.com/olistbr/marketing-funnel-olist?select=olist_marketing_qualified_leads_dataset.csv)
- [E-Commerce Data Set](https://www.kaggle.com/olistbr/brazilian-ecommerce/home?select=product_category_name_translation.csv)
- [Instructions to Link Data Sets](https://www.kaggle.com/andresionek/joining-marketing-funnel-with-brazilian-e-commerce)


In [3]:
import time
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm

In [5]:
df_cd = pd.read_csv('Olist-Dataset/olist_closed_deals_dataset.csv')
df_mql = pd.read_csv('Olist-Dataset/olist_marketing_qualified_leads_dataset.csv')
df_c = pd.read_csv('Olist-Dataset/olist_customers_dataset.csv')
df_g = pd.read_csv('Olist-Dataset/olist_geolocation_dataset.csv')
df_m = pd.read_csv('Olist-Dataset/olist_marketing_qualified_leads_dataset.csv')
df_oi = pd.read_csv('Olist-Dataset/olist_order_items_dataset.csv')
df_op = pd.read_csv('Olist-Dataset/olist_order_payments_dataset.csv')
df_o = pd.read_csv('Olist-Dataset/olist_orders_dataset.csv')
df_or = pd.read_csv('Olist-Dataset/olist_order_reviews_dataset.csv')
df_p = pd.read_csv('Olist-Dataset/olist_products_dataset.csv')
df_s = pd.read_csv('Olist-Dataset/olist_sellers_dataset.csv')
df_t = pd.read_csv('Olist-Dataset/product_category_name_translation.csv')

In [3]:
# Convert spanish and enlish terms columns into lists for product

product_spanish = df_t['product_category_name'].to_list()
product_english = df_t['product_category_name_english'].to_list()
# Replace all spanish terms with english terms
df_pt = df_p.replace(product_spanish,product_english)
# Convert DataFrame into .csv
df_pt.to_csv('Olist-Dataset/olist_products_dataset_english.csv')


In [9]:

# Creating the DataFrame, assigning datatypes. 
# Only considering records:
#   - between order_approved_at dates 2017-01 and 2018-08
#   - orders that have order_status delivered

# Joining Datasets
df_oi_o_c = df_oi.merge(df_o.merge(df_c, on='customer_id', how='inner'), on='order_id', how='inner')

# Converting column datatypes from string to datetime
df_oi_o_c['order_approved_at'] = pd.to_datetime(df_oi_o_c['order_approved_at'])

# Selecting records based on the conditions above
df_oi_o_c = df_oi_o_c.loc[(df_oi_o_c['order_approved_at'] >= datetime.datetime.strptime('2017-01', '%Y-%m')) & (df_oi_o_c['order_approved_at'] < datetime.datetime.strptime('2018-09', '%Y-%m')) & (df_oi_o_c['order_status'] == 'delivered')]

# Dropping Values (Where order_approved_at is null)
nan_index = pd.isnull(df_oi_o_c['order_approved_at']) # to show use: df_oi_o_c.loc[nan_index]
df_oi_o_c = df_oi_o_c.drop(index=df_oi_o_c.loc[nan_index].index.to_list())

# Showing Dataframe
display(df_oi_o_c.sort_values(by=['order_approved_at'], ascending=False))


# Notes:
#     - order_id and order_item_id are a unique set that are a compound key can be proven with: 
#         - d = pd.DataFrame(); d['bar'] = df_oi_o_c.order_id + df_oi_o_c.order_item_id.map(str); d['bar'].unique().shape
      
#     - price and frieght_value spread across order_item_id's when grouped by order_id's does amount to the total prices, as verified.
#         - Example:
#  
#           From olist_order_items_dataset
#           order_id                              order_item_id     price   frieght_value
#           0b223d92c27432930dfe407c6aea3041      1	                209.00	46.48	
#           0b223d92c27432930dfe407c6aea3041	  2	                209.00	46.48

#           From olist_order_payements_dataset
#           order_id                            payment_value
#           0b223d92c27432930dfe407c6aea3041	510.96

#         

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
23530,35a972d7f8436f405b56e36add1a7140,1,d04857e7b4b708ee8b8b9921163edba3,9f505651f4a6abe901a56cdc21508025,2018-08-31 15:10:26,84.99,8.76,898b7fee99c4e42170ab69ba59be0a8b,delivered,2018-08-29 15:00:37,2018-08-29 15:10:26,2018-08-29 16:57:00,2018-08-30 16:23:36,2018-09-05 00:00:00,24ac2b4327e25baf39f2119e4228976a,13483,limeira,SP
1722,03ef5dedbe7492bdae72eec50764c43f,1,c7f27c5bef2338541c772b5776403e6a,7d7866a99a8656a42c7ff6352a433410,2018-08-31 15:05:22,24.90,8.33,496630b6740bcca28fce9ba50d8a26ef,delivered,2018-08-29 14:52:00,2018-08-29 15:05:22,2018-08-29 20:01:00,2018-08-30 16:36:59,2018-09-03 00:00:00,b701bebbdf478f5500348f03aff62121,9541,sao caetano do sul,SP
4924,0b223d92c27432930dfe407c6aea3041,1,2b4472df15512a2825ae86fd9ae79335,67bf6941ba2f1fa1d02c375766bc3e53,2018-08-31 14:30:19,209.00,46.48,e60df9449653a95af4549bbfcb18a6eb,delivered,2018-08-29 14:18:23,2018-08-29 14:31:07,2018-08-29 15:29:00,2018-08-30 16:24:55,2018-09-04 00:00:00,5c58de6fb80e93396e2f35642666b693,80045,curitiba,PR
4925,0b223d92c27432930dfe407c6aea3041,2,2b4472df15512a2825ae86fd9ae79335,67bf6941ba2f1fa1d02c375766bc3e53,2018-08-31 14:30:19,209.00,46.48,e60df9449653a95af4549bbfcb18a6eb,delivered,2018-08-29 14:18:23,2018-08-29 14:31:07,2018-08-29 15:29:00,2018-08-30 16:24:55,2018-09-04 00:00:00,5c58de6fb80e93396e2f35642666b693,80045,curitiba,PR
9829,168626408cb32af0ffaf76711caae1dc,1,bdcf6a834e8faa30dac3886c7a58e92e,2a84855fd20af891be03bc5924d2b453,2018-08-31 14:30:23,45.90,15.39,6e353700bc7bcdf6ebc15d6de16d7002,delivered,2018-08-29 14:18:28,2018-08-29 14:30:23,2018-08-29 18:51:00,2018-08-30 16:52:31,2018-09-11 00:00:00,7febafa06d9d8f232a900a2937f04338,38600,paracatu,MG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101735,e6db6e9529fecbe14cd05dd349816656,1,06ead9c0b05b368667d858c09148af03,b2ba3715d723d245138f291a6fe42594,2017-01-15 21:27:49,109.90,14.94,6b3efc29f67766dd725bb29e857614f2,delivered,2017-01-06 21:27:49,2017-01-06 21:35:20,2017-01-09 12:08:34,2017-01-13 13:57:29,2017-02-20 00:00:00,c9517b423493063fa4e805acf2cc4564,33400,lagoa santa,MG
9408,157ec3dc3f38cdbd2706bd216edfe8fb,1,27066995b777fb84dbcb25961fd6d007,46dc3b2cc0980fb8ec44634e21d2718e,2017-01-10 13:43:16,159.99,15.29,7dfd10dcc726950fc6171cea83872351,delivered,2017-01-06 13:43:16,2017-01-06 13:55:11,2017-01-09 16:03:33,2017-01-13 10:58:13,2017-02-16 00:00:00,48368b31672665cca1b8a03047a1702a,33115,santa luzia,MG
106868,f2dd5f15184c73c0d45c02941c7c23d1,1,b931645cdc2d9868f01544e8db63f5ab,b14db04aa7881970e83ffa9426897925,2017-01-09 22:52:33,65.00,26.92,4b60b3ade055c6ad88a00758c8e8a162,delivered,2017-01-05 22:52:33,2017-01-05 23:05:27,2017-01-06 16:08:45,2017-01-13 17:06:48,2017-02-23 00:00:00,b88b78a413e70182e18b032361b24f91,44900,irece,BA
99649,e1fe072ef14b519af1f0a8ed997c1301,1,743801b34cc44776de511ba8eff778e2,48efc9d94a9834137efd9ea76b065a38,2017-01-09 15:32:59,9.90,14.52,758b633d88b82063db189810084f4ea9,delivered,2017-01-05 15:32:59,2017-01-05 16:15:16,2017-01-06 13:45:22,2017-01-12 14:13:19,2017-02-15 00:00:00,4b3207464f5f7a48a7f63fa0b1251d86,14025,ribeirao preto,SP


In [10]:

# Creating a
periods = [
    '2017-01','2017-02','2017-03','2017-04','2017-05',
    '2017-06','2017-07','2017-08','2017-09','2017-10',
    '2017-11','2017-12','2018-01','2018-02','2018-03',
    '2018-04','2018-05','2018-06','2018-07','2018-08',]
newperiods = periods.copy()
newperiods.append('2018-09')
df = pd.DataFrame(periods, columns=['Periods']).set_index('Periods')

df['total_revenue'] = 0
df['number_of_orders'] = 0                      
df['unique_customers_who_made_a_purchase'] = 0  
df['new_customers'] = 0                 
df['total_customers'] = 0
df['average_customer_lifespan_in_days'] = 0     


# Notes: 
# Total_Revenue: is calculated per month. Counts the total Price including Frieght Value (The dataset does not clarify tax, I will be assuming that there is no tax involved in these sales)

# Number_of_orders: is calculated per month. Does not mutli-count unique orders. Meaning, if multiple purchases are made with one order id, that counts as 1 order.

# Unique_Customers_Who_Made_a_Purchase: is calculated per month. A single Customer is only counted Once per Month, even if a customer purchases mutliple items in a month at different dates.

# New_Customers: is calculated per month. Since the Olist-Dataset does not provide an account creation date. I will be extrapolating that each account was created on the date of the first purchase. To be more exact, the date in which the purchase was approved, 'order_approved_at' in olist_orders_dataset.csv. 

# Total_Customers: is calculated per month. Is essentially a rolling total of New_Customers.

# Average_Customer_Lifespan_in_Days: average customer lifespan = sum of customer lifespans / number of customers.
        # Examples
        # Example 1: 1 Purchase
            # A customer (ID 181) makes one purchase in 09/30/2017 and never purchases again, has a lifespan of 1 day.
            # For the the month of September 2017, the customer has a lifespan of 1 day.
        # Example 2: More than 1 Purchase
            # A customer (ID 182) makes a purchase on 09/30/2017 and 11/30/2017 and ceases to make purchases after. The lifespan is counted only twice. 
            # - The purchase made on 9/30/2017 will be reported in 09/2017 and will have a lifespan of 1 day. 
            # - Since no purchase was made in 10/2017, no reports made
            # - The purchase made on 11/30/2017 will be reported in 11/2017 and will have a lifespan of 2 months (or 60 days).
            # - No other months will count this customer (ID 182) as no other purchases are made


In [315]:
for i in range(len(periods)):
    start_time = datetime.datetime.strptime(periods[i], '%Y-%m')
    if i == 19:
        end_time = datetime.datetime.strptime('2018-09', '%Y-%m')
    else:
        end_time = datetime.datetime.strptime(periods[i+1], '%Y-%m')
    period_df = df_oi_o_c.loc[(df_oi_o_c['order_approved_at'] >= start_time) & (df_oi_o_c['order_approved_at'] < end_time)]
    df.at[periods[i], 'Total_Revenue'] = period_df['price'].sum() + period_df['freight_value'].sum()
    df.at[periods[i], 'Number_of_Orders'] = period_df['order_id'].unique().shape[0]
    df.at[periods[i], 'Unique_Customers_Who_Made_a_Purchase'] = period_df['customer_unique_id'].unique().shape[0]
display(df)
    

,Total_Revenue,Number_of_Orders,Unique_Customers_Who_Made_a_Purchase,Average_Customer_Lifespans
Periods,,,,
2017-01,121884,715,684,0
2017-02,270749,1638,1615,0
2017-03,410734,2554,2516,0
2017-04,387782,2278,2249,0
2017-05,568069,3548,3482,0
2017-06,494351,3143,3084,0
2017-07,560275,3828,3759,0
2017-08,646567,4217,4137,0
2017-09,691353,4170,4103,0


In [12]:
lifetime_df = pd.DataFrame(columns = ['customer_unique_id', '2017-01','2017-02','2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08','2018-09', ])
lifetime_df['customer_unique_id'] = df_oi_o_c['customer_unique_id'].unique()

for i in tqdm(lifetime_df.index):
    id = lifetime_df['customer_unique_id'][i]
    id_df = df_oi_o_c.loc[(df_oi_o_c['customer_unique_id'] == id)][['customer_unique_id','order_approved_at']].reset_index(drop=True)
    id_df['order_approved_at'] = pd.to_datetime(id_df['order_approved_at'])
    for p in range(len(periods)):
        start_time = datetime.datetime.strptime(periods[p], '%Y-%m')
        if p == 19:
            end_time = datetime.datetime.strptime('2018-09', '%Y-%m')
        else:
            end_time = datetime.datetime.strptime(periods[p+1], '%Y-%m')
        period_df = id_df.loc[(id_df['order_approved_at'] >= start_time) & (id_df['order_approved_at'] < end_time)]
        #Good ^^^

        # If a customer (customer_unique_id) has a prior purchase not in the period_df
        if id_df['order_approved_at'].min() not in period_df['order_approved_at'].values:
            lifetime_df.at[i, periods[p]] = (period_df['order_approved_at'].max() - id_df['order_approved_at'].min()).days
        else:
            # Finds the number of days between purchases in a month, if no other purchases had been made prior
            if period_df.shape[0] == 0:         # No purchase, leave as NaN
                pass
            elif period_df.shape[0] == 1:       # 1 purchase, counts as 1 day
                lifetime_df.at[i, periods[p]] = 1
            else:                               # More than 1 purchase in a month, counts the number of days between purchases
                if period_df['order_approved_at'].max() == period_df['order_approved_at'].min(): # For situations where mutliple orders were placed at the same time
                    lifetime_df.at[i, periods[p]] = 1
                elif (period_df['order_approved_at'].max() - period_df['order_approved_at'].min()).days < 1:
                    lifetime_df.at[i, periods[p]] = 1
                else:
                    lifetime_df.at[i, periods[p]] = (period_df['order_approved_at'].max() - period_df['order_approved_at'].min()).days


100%|██████████| 93091/93091 [41:26<00:00, 37.43it/s]


In [14]:
pd.set_option("display.max_columns", 100)
lifetime_df.head(5)

,customer_unique_id,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09
0,871766c5855e863f6eccc05f988b23cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,eb28e67c4c0b83846050ddfb8a35d051,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3818d81c6709e39d06b2738a8d3a2474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,af861d436cfc08b2c2ddefd0ba074622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
4,64b576fb70d441e8f1b2d7d446e483c5,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
new_customers = [lifetime_df.loc[lifetime_df[p]==1].shape[0] for p in periods]
total_customers = []
for i in range(len(new_customers)):
    if i ==0:
        total_customers.append(new_customers[i])
    else:
        total_customers.append(total_customers[i-1] + new_customers[i])
average_customer_lifespan_in_days = []
for i, p in enumerate(periods):
	average_customer_lifespan_in_days.append(lifetime_df[p].sum()/total_customers[i])
average_customer_lifespan_in_days

[1.0029282576866765,
 0.7456293706293706,
 0.5635036496350365,
 0.41642938496583143,
 0.4914897685982023,
 0.4166481357942332,
 0.44097282830045964,
 0.4031521994824747,
 0.42421723469105016,
 0.43205740007445764,
 0.5656050256494665,
 0.42833973334605385,
 0.5648252768247838,
 0.4069839420310964,
 0.4444300518134715,
 0.4532558787718758,
 0.4888087125008352,
 0.46845027138219036,
 0.3880388304634231,
 0.4306450570055389]

In [ ]:
df['New_Customers'] = new_customers
df['Total_Customers'] = total_customers
df = df.drop(['Average_Customer_Lifespans'], axis=1)
df['Average_Customer_Lifespan_in_Days'] = average_customer_lifespan_in_days

In [20]:
df.to_excel('Insight-Dataset/KPI-Dataset.xlsx', engine='openpyxl')

[683,
 2288,
 4795,
 7024,
 10458,
 13491,
 17187,
 21255,
 25263,
 29547,
 36453,
 41927,
 48677,
 54926,
 61760,
 68169,
 74835,
 80698,
 86530,
 92798]